In [1]:
from glob import glob
import re
import json
def loadFile(file_name) :
    with open(file_name, "r") as f :
        return "".join(f.readlines())
    return ""

def saveFile(file_name, contents) :
    with open(file_name, "w") as f :
        f.write(contents)

def loadJson(json_file_path) :
    with open(json_file_path, encoding='utf-8') as json_file :
        return json.load(json_file)

def camel2Sname(name) :
    return re.sub(r'(?<!^)(?=[A-Z])', '_', name).lower()

def camel2SnameSpace(name) :
    return re.sub(r'(?<!^)(?=[A-Z])', ' ', name).lower()


simpleApi = loadJson("../ngx3js-module/docs_api.json")["children"]


In [2]:
def getClassInfo(className : str) :
    for info in simpleApi :
        if (info["name"] == className) :
            return info
    return None

def getSourceFile(info) :
    if "sources" in info :
        fileName : str = info["sources"][0]["fileName"]
        if fileName.startswith("src/lib/") :
            return fileName[8:-3]
    return "error"

def getSameSourceFile(sorceUrl) :
    list = []
    for info in simpleApi :
        infoSource = getSourceFile(info)
        kindString = info["kindString"]
        if sorceUrl == infoSource and kindString != 'Class':
           list.append(info) 
    return list

def isDecorators(info, decoType) :
    isInput = False
    if 'decorators' in info :
        decorators = info['decorators']
        for deco in decorators :
            if deco['name'] == decoType :
                isInput = True
    return isInput

def getExtends(info) :
    extendeds = []
    if "extendedTypes" in info : 
        extendedTypeName = info["extendedTypes"][0]["name"]
        childInfo = getClassInfo(extendedTypeName)
        if childInfo is not None :
            parentExtends = getExtends(childInfo)
            if len(parentExtends) > 0 :
                for name in parentExtends :
                    extendeds.append(name)
        extendeds.append(extendedTypeName)
    return extendeds


def getParamTypes(child) :
    typeName = 'any'
    if 'type' in child and 'type' in child['type']:
        childType = child['type']['type']
        if childType == 'union' :
            typeNames = []
            for typeInfo in child['type']['types'] :
                if 'name' in typeInfo :
                    typeNames.append(typeInfo['name'])
                elif typeInfo['type'] == 'literal' and 'value' in typeInfo:
                    typeNames.append("'" + typeInfo['value'] + "'")
                elif typeInfo['type'] == 'array' and 'elementType' in typeInfo and 'name' in typeInfo['elementType']:
                    typeNames.append(typeInfo['elementType']['name'] + "[]")
            typeName = '|'.join(typeNames)
            # typeName = 'mixed'
        elif childType == 'array' : 
            if 'elementType' in child['type'] and 'name' in child['type']['elementType'] :
                typeName = child['type']['elementType']['name'] + "[]"
            #else  :
                #print("Error")
                #print(child['type'])
        elif 'name' in child['type']:
            typeName = child['type']['name']
    return typeName

def getGenApiDocs(className : str) :
    info = getClassInfo(className)
    if info is None :
        return False
    kindString = info["kindString"]
    html = []
    html.append('<!DOCTYPE html>')
    html.append('<html lang="en">')
    html.append('	<head>')
    html.append('		<meta charset="utf-8" />')
    html.append('		<base href="../../" />')
    html.append('		<script src="page.js"></script>')
    html.append('		<link type="text/css" rel="stylesheet" href="page.css" />')
    html.append('	</head>')
    html.append('	<body>')
    extends = getExtends(info)
    if len(extends) > 0 :
        extendHtml = []
        for ext in extends :
            extendHtml.append("[page:{0}] &rarr;".format(ext))
        html.append('		' + " ".join(extendHtml))
        html.append('')
    html.append('		<h1>{0}</h1>'.format(className))
    html.append('')
    html.append('		<p class="desc">\n\t\t\t{0}\n\t\t</p>'.format(getComment(info)))
    html.append('')
    fileName = getSourceFile(info)
    if kindString == "Class" :
        html.append('		<h2>Constructor</h2>')
        methodCnt = 0
        for child in info['children'] :        
            sources = getSourceFile(child)
            kindString = child['kindString']
            flags = child['flags']
            if sources == fileName and (kindString == 'Constructor'):
                params = []
                returnType  = 'void' 
                comment = None
                paramsComments = []
                funcModifier = []
                if 'isPublic' in flags and flags['isPublic'] :
                    funcModifier.append('<i class="modifier modifier-public">Public</i>')
                elif 'isPrivate' in flags and flags['isPrivate'] :
                    funcModifier.append('<i class="modifier modifier-private">Private</i>')
                if 'isProtected' in flags and flags['isProtected'] :
                    funcModifier.append('<i class="modifier modifier-protected">Protected</i>')
                if 'isStatic' in flags and flags['isStatic'] :
                    funcModifier.append('<i class="modifier modifier-static">Static</i>')
                if 'overwrites' in child :
                    funcModifier.append('<i class="modifier modifier-overwrite">Overwrite</i>')
                funcModifierStr = ' '.join(funcModifier)
                if 'signatures' in child :
                    for signatures in child['signatures'] :
                        returnType = getReturnType(signatures)
                        comment = getComment(signatures)
                        if 'parameters' in signatures :
                            for parameters in signatures['parameters'] :
                                typeName = getParamTypes(parameters)
                                isOptional = ''
                                paramComment = ''
                                isOptionalComment = ''
                                if 'flags' in parameters and 'isOptional' in parameters['flags'] and  parameters['flags']['isOptional'] :
                                    isOptional = '?'
                                    isOptionalComment = '(optional) '
                                if 'comment' in parameters and 'shortText' in parameters['comment'] :
                                    paramComment = parameters['comment']['shortText'].strip()
                                params.append(" [param:{0} {1}]{2}".format(typeName, parameters['name'], isOptional)); 
                                if paramComment == '' :
                                    paramComment = "The {0}.".format(camel2SnameSpace(parameters['name']))
                                paramsComments.append("[parameter:{0} {1}] - {2}{3}".format(typeName, parameters['name'], isOptionalComment, paramComment)); 
                html.append('		<h3>new [method:{1} {0}]( {2} ){3}</h3>'.format(
                    className, 
                    returnType,
                    ", ".join(params),
                    funcModifierStr
                ))
                if len(paramsComments) > 0 :
                    html.append('		<h4>Parameters</h4>\n\t\t<p>\n\t\t\t{0}\n\t\t</p>'.format("<br />\n\t\t\t".join(paramsComments)))
                if comment is not None :
                    html.append('		<p>\n\t\t\t{0}\n\t\t</p>'.format(comment))
                html.append('')
                methodCnt += 1
        if methodCnt <= 0 :
            html.append('		<p>\n\t\t\tNone\n\t\t</p>')
            html.append('')

        if "children" in info:
            if isDecorators(info, 'Component') :
                decorators = info['decorators']
                for deco in decorators :
                    name = deco['name']
                    if name == 'Component' :
                        selector = re.compile("selector: '([a-z0-9\-]+)'")
                        matches = selector.findall(deco['arguments']['obj'])
                        if matches :
                            tagName = matches[0]
                            html.append('		<h3>Angular Html Template [ Input ] ( Output )</h3>'.format(deco['name']))
                            html.append('		<code data-type="html">&lt;{0}'.format(tagName))
                            for child in info['children'] :        
                                kindString = child['kindString']
                                paramType = getParamTypes(child)
                                if kindString == 'Property' :
                                    attrName = child['name']
                                    shortText = ''
                                    if 'comment' in child and 'shortText' in child['comment'] :
                                        shortText = child['comment']['shortText'].replace('\t',' ').replace('\n',' ').replace('   ',' ').replace('  ',' ').replace('  ',' ').strip()
                                        if shortText != '' and len(shortText) > 30:
                                            shortText = shortText[0:30] + '...'
                                        if shortText != '' :
                                            shortText = ' -- {0} '.format(shortText) 
                                    if isDecorators(child, 'Input') :
                                        html.append('			[ {0} ]="... {1}{2}"'.format(attrName, paramType, shortText))
                                    elif isDecorators(child, 'Output') :
                                        typeArguments = 'any'
                                        if 'type' in child and 'typeArguments' in child['type'] and len(child['type']['typeArguments']) > 0 :
                                            typeArguments = child['type']['typeArguments'][0]['name']
                                        html.append('			( {0} )="...($event : {1}{2})"'.format(attrName, typeArguments, shortText))
                            html.append('		&gt;&lt;/{0}&gt;'.format(tagName))
                            html.append('		</code>')
                            html.append('')

            html.append('		<h2>Properties</h2>')
            inputCnt = 0
            if len(extends) > 0 :
                for ext in extends :
                    html.append('		<p>')
                    html.append('			See the base [page:{0} {0}] class for common properties.'.format(ext))
                    html.append('		</p>')
                    inputCnt += 1
                html.append('')
            for child in info['children'] :        
                sources = getSourceFile(child)
                kindString = child['kindString']
                flags = child['flags']
                if sources == fileName and kindString == 'Property' and (isDecorators(child, 'Input') or isDecorators(child, 'Output') or ('isPublic' in flags and flags['isPublic']) or ('isProtected' in flags and flags['isProtected'])):
                    typeName = getParamTypes(child)
                    isOptional = ''
                    funcModifier = []
                    if isDecorators(child, 'Input') :
                        funcModifier.append('<i class="modifier modifier-input">Input</i>')
                    if isDecorators(child, 'Output') :
                        funcModifier.append('<i class="modifier modifier-output">Output</i>')
                    if 'isPublic' in flags and flags['isPublic'] :
                        funcModifier.append('<i class="modifier modifier-public">Public</i>')
                    elif 'isPrivate' in flags and flags['isPrivate'] :
                        funcModifier.append('<i class="modifier modifier-private">Private</i>')
                    if 'isProtected' in flags and flags['isProtected'] :
                        funcModifier.append('<i class="modifier modifier-protected">Protected</i>')
                    if 'isStatic' in flags and flags['isStatic'] :
                        funcModifier.append('<i class="modifier modifier-static">Static</i>')
                    if 'flags' in child and 'isOptional' in child['flags'] and  child['flags']['isOptional'] :
                        isOptional = '?'
                        funcModifier.append('<i class="modifier modifier-optional">Optional</i>')
                    funcModifierStr = ' '.join(funcModifier)
                    defaultValue = ''
                    if 'defaultValue' in child and child['defaultValue'] != 'null':
                        defaultValue = ' &nbsp; = {0} '.format(child['defaultValue'])
                    html.append('		<h3>[property:{1} {0}]{2}{4}{3}</h3>'.format(
                        child['name'], 
                        typeName,
                        isOptional,
                        funcModifierStr,
                        defaultValue
                    ))
                    html.append('		<p>\n\t\t\t{0}\n\t\t</p>'.format(getComment(child)))
                    html.append('')
                    inputCnt += 1
            if inputCnt <= 0 :
                html.append('		<p>\n\t\t\tNone\n\t\t</p>')
                html.append('')
            html.append('		<h2>Methods</h2>')
            methodCnt = 0
            if len(extends) > 0 :
                for ext in extends :
                    html.append('		<p>')
                    html.append('			See the base [page:{0} {0}] class for common methods.'.format(ext))
                    html.append('		</p>')
                    methodCnt += 1
                html.append('')
            for child in info['children'] :        
                sources = getSourceFile(child)
                kindString = child['kindString']
                flags = child['flags']
                if sources == fileName and (kindString == 'Method') and (
                    ('isPublic' in flags and flags['isPublic']) or ('isProtected' in flags and flags['isProtected'])) :
                    params = []
                    returnType  = 'void' 
                    comment = None
                    paramsComments = []
                    funcModifier = []
                    if 'isPublic' in flags and flags['isPublic'] :
                        funcModifier.append('<i class="modifier modifier-public">Public</i>')
                    elif 'isPrivate' in flags and flags['isPrivate'] :
                        funcModifier.append('<i class="modifier modifier-private">Private</i>')
                    if 'isProtected' in flags and flags['isProtected'] :
                        funcModifier.append('<i class="modifier modifier-protected">Protected</i>')
                    if 'isStatic' in flags and flags['isStatic'] :
                        funcModifier.append('<i class="modifier modifier-static">Static</i>')
                    if 'overwrites' in child :
                        funcModifier.append('<i class="modifier modifier-overwrite">Overwrite</i>')
                    funcModifierStr = ' '.join(funcModifier)
                    if 'signatures' in child :
                        for signatures in child['signatures'] :
                            returnType = getReturnType(signatures)
                            comment = getComment(signatures)
                            if 'parameters' in signatures :
                                for parameters in signatures['parameters'] :
                                    typeName = getParamTypes(parameters)
                                    isOptional = ''
                                    paramComment = ''
                                    isOptionalComment = ''
                                    if 'flags' in parameters and 'isOptional' in parameters['flags'] and  parameters['flags']['isOptional'] :
                                        isOptional = '?'
                                        isOptionalComment = '(optional) '
                                    if 'comment' in parameters and 'shortText' in parameters['comment'] :
                                        paramComment = parameters['comment']['shortText'].strip()
                                    params.append(" [param:{0} {1}]{2}".format(typeName, parameters['name'], isOptional)); 
                                    if paramComment == '' :
                                        paramComment = "The {0}.".format(camel2SnameSpace(parameters['name']))
                                    paramsComments.append("[parameter:{0} {1}] - {2}{3}".format(typeName, parameters['name'], isOptionalComment, paramComment)); 
                    html.append('		<h3>[method:{1} {0}]( {2} ){3}</h3>'.format(
                        child['name'], 
                        returnType,
                        ", ".join(params),
                        funcModifierStr
                    ))
                    if len(paramsComments) > 0 :
                        html.append('		<h4>Parameters</h4>\n\t\t<p>\n\t\t\t{0}\n\t\t</p>'.format("<br />\n\t\t\t".join(paramsComments)))
                    if comment is not None :
                        html.append('		<p>\n\t\t\t{0}\n\t\t</p>'.format(comment))
                    html.append('')
                    methodCnt += 1
            if methodCnt <= 0 :
                html.append('		<p>\n\t\t\tNone\n\t\t</p>')
                html.append('')

    if 'sources' in info :
        sourcesPath = info['sources'][0]['fileName']
        refClassList = getSameSourceFile(getSourceFile(info))
        for mchild in refClassList :
            extends = getExtends(mchild)
            if len(extends) > 0 :
                extendHtml = []
                for ext in extends :
                    extendHtml.append("[page:{0}] &rarr;".format(ext))
                html.append('		' + " ".join(extendHtml))
                html.append('')
            mChildName = mchild['name']
            html.append('		<hr class="interface-line"/>')
            html.append('		<h3 id="{0}">{0} <i class="modifier modifier-interface">Interface</i></h3>'.format(mChildName))
            getExtraInterFace(html, mchild, extends)
            html.append('')
            html.append('		<p class="desc">\n\t\t\t{0}\n\t\t</p>'.format(getComment(mchild)))
            html.append('')
        html.append('		<br /><br />')
        html.append('		<h2>Source</h2>')
        html.append('		<p>[link:https://github.com/outmindkjg/ngx3js-module/tree/master/projects/ngx3js/src/lib{0}.ts {0}.ts]</p>'.format(sourcesPath[7:-3]))
    html.append('	</body>')
    html.append('</html>')
    saveFile('src/assets/ngxapi/en/{0}.html'.format(className),"\n".join(html))
    return True

def getCommentCode(comment : str) :
    # print(comment)
    restComment = comment
    restComment = restComment.replace('>','&gt;').replace('<','&lt;')
    if restComment is not None and restComment.find("```") > -1 :
        while restComment.find("```") > -1 :
            start = restComment.find("```")
            end = restComment.find("```", start + 3)
            blockType = 'text'
            conStart = start + 3
            if restComment[conStart: conStart+2] == 'ts' :
                conStart += 2
                blockType = 'typescript'
            elif restComment[conStart: conStart+4] == 'html' :
                conStart += 4
                blockType = 'html'
            elif restComment[conStart: conStart+4] == 'text' :
                conStart += 4
                blockType = 'text'
            planeHtml = restComment[conStart: end]
            planeHtml = planeHtml.replace('>','&gt;')
            planeHtml = planeHtml.replace('<','&lt;')
            restComment = restComment[0: start] + '<code data-type="'+blockType+'">' +  planeHtml + '</code>' + restComment[end + 3:]
    return restComment

def getReturnType(signatures) :
    returnType = 'void'
    if 'type' in signatures :
        if 'name' in signatures['type'] :
            returnType = signatures['type']['name']
        elif 'type' in signatures['type'] :
            childType = signatures['type']['type']
            if childType == 'union' and 'types' in signatures['type'] :
                typeNames = []
                for typeInfo in signatures['type']['types'] :
                    if 'name' in typeInfo :
                        typeNames.append('{0}'.format(typeInfo['name']))
                    elif typeInfo['type'] == 'array' and 'elementType' in typeInfo and 'name' in typeInfo['elementType']:
                        typeNames.append('{0}[]'.format(typeInfo['elementType']['name']))
                if len(typeNames) > 0 :
                    returnType = "|".join(typeNames).strip()
            elif childType == 'array' and 'elementType' in signatures['type'] and 'name' in signatures['type']['elementType'] :
                returnType = '{0}[]'.format(signatures['type']['elementType']['name'])

    return returnType

def getExtraInterFace(html : list[str], mchild, extends) :
    fileName = getSourceFile(mchild)
    kindString = mchild["kindString"]
    if kindString == "Interface" :
        html.append('		<h4>Properties</h4>')
        propertiesCnt = 0
        if len(extends) > 0 :
            for ext in extends :
                html.append('		<p>')
                html.append('			See the base [page:{0} {0}] class for common properties.'.format(ext))
                html.append('		</p>')
                propertiesCnt += 1
            html.append('')
        if "children" in mchild:
            properties = []
            for child in mchild['children'] :        
                sources = getSourceFile(child)
                kindString = child['kindString']
                if sources == fileName and kindString == 'Property' :
                    typeName = getParamTypes(child)
                    isOptional = ''
                    if 'flags' in child and 'isOptional' in child['flags'] and  child['flags']['isOptional'] :
                        isOptional = '?'
                    paramComment = getComment(child)
                    if paramComment is None or paramComment == '' :
                        paramComment = ' The {0}.'.format(camel2SnameSpace(child['name']))
                    defaultValue = ''
                    if 'defaultValue' in child :
                        defaultValue = ' = {0}'.format(child['defaultValue'])
                    properties.append('[property:{1} {0}]{2} - {3}'.format(
                        child['name'], 
                        typeName,
                        isOptional,
                        paramComment,
                        defaultValue
                    ))
                    propertiesCnt += 1
            html.append('		<p>\n\t\t\t{0}\n\t\t</p>'.format('<br />\n\t\t\t'.join(properties)))
            html.append('')
        if propertiesCnt <= 0 :
            html.append('		<p>\n\t\t\tNone\n\t\t</p>')
            html.append('')

    elif kindString == "Type alias" or kindString == 'Variable':
        typeInfo = mchild['type']
        typeName = typeInfo['type']
        if typeName == 'reflection' :
            params = []
            returnType  = 'void' 
            comment = None
            child = typeInfo['declaration']
            if 'signatures' in child :
                for signatures in child['signatures'] :
                    returnType = getReturnType(signatures)
                    comment = getComment(signatures)
                    properties = []
                    if 'parameters' in signatures :
                        for parameters in signatures['parameters'] :
                            typeName = getParamTypes(parameters)
                            isOptional = ''
                            if 'flags' in parameters and 'isOptional' in parameters['flags'] and  parameters['flags']['isOptional'] :
                                isOptional = '?'
                            params.append(" [param:{0} {1}]{2}".format(typeName, parameters['name'], isOptional)); 
                            paramComment = getComment(parameters)
                            if paramComment is None or paramComment == '' :
                                paramComment = ' The {0}.'.format(camel2SnameSpace(parameters['name']))
                            properties.append('[property:{1} {0}]{2} - {3}'.format(
                                parameters['name'], 
                                typeName,
                                isOptional,
                                paramComment
                            ))
                childName = child['name']
                if childName == '__type':
                    childName = 'function'
                html.append('		<h4>[method:{1} {0}]( {2} )</h4>'.format(
                    childName, 
                    returnType,
                    ", ".join(params)
                ))
                if len(properties) > 0 :
                    html.append('		<h4>Parameters</h4>')
                    html.append('		<p>\n\t\t\t{0}\n\t\t</p>'.format('<br />\n\t\t\t'.join(properties)))
            if 'indexSignature' in child :
                signatures = child['indexSignature']
                returnType = getReturnType(signatures)
                comment = getComment(signatures)
                if 'parameters' in signatures :
                    for parameters in signatures['parameters'] :
                        typeName = getParamTypes(parameters)
                        isOptional = ''
                        if 'flags' in child and 'isOptional' in child['flags'] and  child['flags']['isOptional'] :
                            isOptional = '?'
                        params.append(" [param:{0} {1}]{2}".format(typeName, parameters['name'], isOptional)); 
                html.append('		<h3> {{ [ {2} ] : [param:{1} emptyName] }}</h3>'.format(
                    child['name'], 
                    returnType,
                    ", ".join(params)
                ))
            
            if comment is not None :
                html.append('		<p>\n\t\t\t{0}\n\t\t</p>'.format(comment))
            html.append('')
        elif typeName == 'union' :
            child = mchild
            typeName = getParamTypes(child)
            html.append('		<h3>[property:{1} {0}]</h3>'.format(
                child['name'], 
                typeName
            ))
        html.append('')
    else :
        print(kindString)    
    return html

def getComment(info) :
    commentHtml = ''
    comment = None
    commentTxt = []
    funcReturns = ''
    shortText = ''
    if "comment" in info :
        comment = info['comment']
    if comment is not None :
        if 'shortText' in comment :
            shortText = getCommentCode(comment['shortText'].strip()) + "\n"
        if 'text' in comment :
            commentTxt.append(getCommentCode(comment['text'].strip()))
        if 'tags' in comment :
            tagHtml = []
            for tags in comment['tags'] :
                if (tags['tag'] == 'see'):
                    tagText = tags['text'].strip()
                    if tagText.endswith(",") : 
                        tagText = tagText[0:-1]
                    threeSelector = re.compile("THREE\.([a-zA-Z0-9]+)")
                    matches = threeSelector.findall(tagText)
                    if matches :
                        tagHtml.append("[page:{0} {0}] - {1}".format(
                            matches[0],
                            tagText
                        ))
                    else :
                        linkSelector = re.compile("{@link (.+)\|(.+)\}")
                        matches = linkSelector.findall(tagText)
                        if matches :
                            linkUrl = matches[0][0].strip()
                            linkName = matches[0][1].strip()
                            if linkUrl.startswith('https://outmindkjg.github.io/ngx3js-doc/#/examples/') : 
                                tagHtml.append("[example:{0} {1}] ".format(
                                    linkUrl[51:],
                                    linkName
                                ))
                            else :
                                tagHtml.append("[link:{0} {1}]".format(
                                    linkUrl,
                                    linkName
                                ))
            if len(tagHtml) > 0 :
                commentTxt.append("")
                commentTxt.append("\n".join(tagHtml).strip())
    if 'type' in info and 'type' in info['type']['type']:
        childType = info['type']['type']
        if childType == 'union' :
            typeNames = []
            for typeInfo in info['type']['types'] :
                if 'name' in typeInfo :
                    typeNames.append('[parameter:{0} {0}]'.format(typeInfo['name']))
                elif typeInfo['type'] == 'array' and 'elementType' in typeInfo and 'name' in typeInfo['elementType']:
                    typeNames.append('[parameter:{0} {0}]'.format(typeInfo['elementType']['name']))
            if len(typeNames) > 0 :
                commentTxt.append("")
                commentTxt.append(" | ".join(typeNames).strip())
        elif childType == 'reflection' and 'declaration' in info['type']:
            declaration = info['type']['declaration']
            if 'indexSignature' in declaration :
                parameters = declaration['indexSignature']['parameters']
                key = parameters[0]['name']
                keyType = parameters[0]['type']['name']
                value = 'any'
                if 'name' in declaration['indexSignature']['type'] :
                    value = declaration['indexSignature']['type']['name']
                commentTxt.append("")
                commentTxt.append(" {{ [ {0} : [page:{1} {1}] ] : [page:{2} {2}] }} ".format(key, keyType, value ))
    funcComment = ''
    if comment is not None :
        if 'returns' in comment :
            funcComment = comment['returns'].strip()

    if funcComment != '' or funcReturns != '':
        commentTxt.append("")
        commentTxt.append("Returns : {0} {1}".format(funcComment, funcReturns ))
    
    if len(commentTxt) == 0 :
        commentTxt.append(shortText)
        shortText = ''
    if len(commentTxt) > 0 :
        commentHtml = "\n".join(commentTxt)
        commentHtml = commentHtml.replace("_@Component","@Component").replace("_@Directive","@Directive").replace("_@Pipe","@Pipe")
        commentHtml = commentHtml.replace("<br>","").replace("<br/>","").replace("<br />","")
        commentHtml = commentHtml.replace("\n\n\n","\n\n")
        commentHtml = re.sub('\[([^\]]+)\]\(([^\)]+)\)', r'[link:\2 \1]', commentHtml)
        commentHtml = re.sub('\[link:https://outmindkjg.github.io/ngx3js-doc/#/docs/[a-z0-9\/]+/([a-zA-Z0-9\.]+) ', r'[page:\1 ', commentHtml)
        commentHtml = re.sub('\[link:https://outmindkjg.github.io/ngx3js-doc/#/examples/([a-zA-Z_0-9\.]+) ', r'[example:\1 ', commentHtml)
        previewSelector = re.compile("\[example:([A-Za-z_0-9]+) ([^\]]+)\]")
        matches = previewSelector.findall(commentHtml)
        if matches :
            linkUrl = matches[0][0].strip()
            linkName = matches[0][1].strip()
            commentHtml =  shortText + '\n<h3>Preview - {0}</h3>\n<iframe src="#/viewer/{1}"></iframe>\n\n'.format(linkName, linkUrl) + commentHtml
        else :
            commentHtml =  shortText + commentHtml
        commentHtml = commentHtml.strip().replace("\n","<br />\n\t\t\t").strip()

    return commentHtml


In [28]:
# getGenApiDocs("MeshComponent")
# getGenApiDocs("AbstractGeometryComponent")
# getGenApiDocs("GeometryComponent")


getGenApiDocs("GeometryComponent")
getGenApiDocs("AudioComponent")


True

In [3]:

loadedClass = {}
jsonList = []
for child in simpleApi :
    name = child["name"]
    kindString = child["kindString"]
    if name not in loadedClass and kindString == 'Class':
        loadedClass[name] = child['sources'][0]['fileName']

for name in loadedClass :
    url = loadedClass[name]
    # print(url)
    if url.startswith('src/lib/') :
        if getGenApiDocs(name) :
            jsonList.append('"{0}" : "ngxapi/en/{0}"'.format(name)) 
print(",\n".join(jsonList))

"AbstractChartComponent" : "ngxapi/en/AbstractChartComponent",
"AbstractControllerComponent" : "ngxapi/en/AbstractControllerComponent",
"AbstractGeometryComponent" : "ngxapi/en/AbstractGeometryComponent",
"AbstractMaterialComponent" : "ngxapi/en/AbstractMaterialComponent",
"AbstractObject3dComponent" : "ngxapi/en/AbstractObject3dComponent",
"AbstractObject3dDirective" : "ngxapi/en/AbstractObject3dDirective",
"AbstractSubscribeComponent" : "ngxapi/en/AbstractSubscribeComponent",
"AbstractTextureComponent" : "ngxapi/en/AbstractTextureComponent",
"AbstractThreeController" : "ngxapi/en/AbstractThreeController",
"AbstractThreeDirective" : "ngxapi/en/AbstractThreeDirective",
"AbstractTweenComponent" : "ngxapi/en/AbstractTweenComponent",
"AnglePipe" : "ngxapi/en/AnglePipe",
"AnimationGroupComponent" : "ngxapi/en/AnimationGroupComponent",
"AssetsPipe" : "ngxapi/en/AssetsPipe",
"AudioComponent" : "ngxapi/en/AudioComponent",
"AutoMaterialController" : "ngxapi/en/AutoMaterialController",
"AutoPos